In [4]:
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# add the path to my packages to system paths so they can be imported
import sys
sys.path.append('/home/yasamanparhizkar/Documents/yorku/01_thesis/code/my_packages')
# sys.path.append('F:\MAScThesis\code\my_packages')
# sys.path.append('/home/yasamanparhizkar/Documents/thesis/code/my_packages')

import dataprocess.data_handler_01 as dh
warnings.filterwarnings("ignore")

In [5]:
diamonds = sns.load_dataset("diamonds")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [6]:
type(diamonds)

pandas.core.frame.DataFrame

In [7]:
print(diamonds)

       carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


# Create a pandas dataframe from fish movie features

In [8]:
# load all spike data from file
spikes_dp = '../../data/original_files/spikes.csv'
binned_data = np.loadtxt(spikes_dp, delimiter=',')
binned_data = binned_data.reshape(binned_data.shape[0], 1141, 113)
binned_data = binned_data * 2 - 1     # turn labels from 0,1 to -1,1

I_order_10 = [54, 35, 10, 60, 74, 9, 61, 56, 91, 104]

# group all neurons together
grouped_data = np.zeros((297, 1141, 1))
for trial in range(297):
    for frame in range(1141):
        grouped_data[trial, frame, :] = 2 * int((binned_data[trial, frame, :] == 1).any()) - 1
        
print('grouped_data.shape = ', grouped_data.shape)

grouped_data.shape =  (297, 1141, 1)


In [9]:
# use only the second trial
ind_min = 1*1141+0
ind_max = 2*1141-1

def transform(fv):
    """
    Transform to be applied on feature vectors.
    
    Input: fv
    fv - 1xDf torch tensor representing a feature vector
    
    Output: fvv
    fvv - 1xDf' torch tensor representing the transformed feature vector
    """
    
    # for faster run and less memory usage
    fvv = fv[:, ::10]
    
    # for numerical stability during GD
    # fvv = fvv * 10
    
    return fvv

data_params = {'func': dh.datapoint_torch, 'features_dp': '../../data/features/slowfast/slowfast_4732/', \
               'spike_data': grouped_data, 'group_id': 0, 'transform': transform}

train_num = 500
val_num = 100
seed = None

# create training and validation datasets
train_num, val_num, train_data, val_data = dh.random_train_val(train_num, val_num, ind_min, ind_max, data_params, seed)

print('# training datapoints: ', train_num)
print('# validation datapoints: ', val_num)
print('features array shape (NxDf): ', train_data['des'].shape)
print('a few rows of the training feature array:\n',train_data['des'][:5])

# training datapoints:  500
# validation datapoints:  100
features array shape (NxDf):  (500, 474)
a few rows of the training feature array:
 [[2.7729063 1.8631061 1.9133807 ... 2.6633606 5.0908637 2.8500423]
 [2.8124714 1.8130212 1.8859538 ... 2.5953789 5.0389576 3.9456177]
 [2.760902  1.9105086 1.982268  ... 2.6250732 5.0870233 1.9213662]
 [2.7647893 1.9004955 2.0001106 ... 2.604283  4.988652  1.6667227]
 [2.8000011 1.9580019 1.8764307 ... 2.6044118 5.1214724 1.7195414]]


In [10]:
# create a dataframe from the features array
train_df = pd.DataFrame(data = train_data['des'])
val_df = pd.DataFrame(data = val_data['des'])
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,464,465,466,467,468,469,470,471,472,473
0,2.772906,1.863106,1.913381,3.041141,2.811475,7.394444,4.027784,5.446352,5.462409,3.120490,...,3.928102,7.577425,2.168741,4.612000,2.208712,5.237152,4.845102,2.663361,5.090864,2.850042
1,2.812471,1.813021,1.885954,3.140523,2.846694,7.458174,4.050166,5.450582,5.401821,3.692590,...,2.795425,3.832484,2.227428,4.710648,2.184887,2.849273,3.836781,2.595379,5.038958,3.945618
2,2.760902,1.910509,1.982268,3.089719,2.766230,7.346116,4.204958,5.519397,5.470970,3.035503,...,2.151127,2.690541,1.995130,4.764640,2.182679,2.367900,1.956508,2.625073,5.087023,1.921366
3,2.764789,1.900496,2.000111,3.300560,2.789424,7.466080,6.823022,5.366567,5.522698,3.059856,...,2.307855,2.787922,2.246354,4.753386,2.238392,2.398641,1.865403,2.604283,4.988652,1.666723
4,2.800001,1.958002,1.876431,3.177871,2.831660,7.387920,4.051704,4.782259,5.445812,2.999777,...,2.220134,2.756427,2.385973,4.765700,2.164595,2.410181,2.021915,2.604412,5.121472,1.719541


In [11]:
# create pandas series from the labels
train_y = pd.DataFrame(data = train_data['lbls'])
val_y = pd.DataFrame(data = val_data['lbls'])
train_y.head()

,0
0,1.0
1,1.0
2,-1.0
3,-1.0
4,1.0


In [12]:
# create xgb Dmatrices from dataframes
dtrain_from_df = xgb.DMatrix(train_df, train_y)
print('Dmatrix shape: {} x {}'.format(dtrain_from_df.num_row(), dtrain_from_df.num_col()))
print('Dmatrix labels: ', dtrain_from_df.get_label())

Dmatrix shape: 500 x 474
Dmatrix labels:  [ 1.  1. -1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.
  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1.  1.
 -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1. -1. -1. -1.
  1. -1.  1. -1.  1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.
  1. -1. -1. -1.  1.  1. -1.  1. -1. -1. -1.  1.  1.  1. -1.  1.  1. -1.
  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.
  1.  1.  1. -1. -1.  1. -1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1.  1.
  1.  1. -1.  1.  1. -1.  1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1.  1.
 -1. -1. -1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.
 -1.  1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1. -1.  1.
 -1.  1. 

In [26]:
# create xgb Dmatrices directly from numpy arrays
dtrain = xgb.DMatrix(train_data['des'], label=(train_data['lbls']+1)//2)
dval = xgb.DMatrix(val_data['des'], label=(val_data['lbls']+1)//2)

In [27]:
print('Train Dmatrix shape: {} x {}'.format(dtrain.num_row(), dtrain.num_col()))
print('Val Dmatrix shape: {} x {}'.format(dval.num_row(), dval.num_col()))
print('Train Dmatrix labels: ', dtrain.get_label())
print('Val Dmatrix labels: ', dval.get_label())

Train Dmatrix shape: 500 x 474
Val Dmatrix shape: 100 x 474
Train Dmatrix labels:  [1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0.
 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1

# Training

In [28]:
# Define hyperparameters
params = {"objective": "binary:logistic", "tree_method": "hist"}
evals = [(dtrain, "train"), (dval, "validation")]
n = 20
veval = 1

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=veval # print losses every ten rounds
)

[0]	train-logloss:0.51791	validation-logloss:0.58426
[1]	train-logloss:0.41573	validation-logloss:0.52100
[2]	train-logloss:0.34269	validation-logloss:0.49901
[3]	train-logloss:0.27600	validation-logloss:0.46496
[4]	train-logloss:0.23084	validation-logloss:0.46228
[5]	train-logloss:0.19491	validation-logloss:0.43869
[6]	train-logloss:0.16608	validation-logloss:0.43089
[7]	train-logloss:0.13953	validation-logloss:0.42889
[8]	train-logloss:0.12093	validation-logloss:0.41069
[9]	train-logloss:0.10580	validation-logloss:0.39907
[10]	train-logloss:0.09175	validation-logloss:0.40084
[11]	train-logloss:0.08038	validation-logloss:0.40522
[12]	train-logloss:0.07248	validation-logloss:0.40027
[13]	train-logloss:0.06550	validation-logloss:0.39856
[14]	train-logloss:0.05828	validation-logloss:0.39732
[15]	train-logloss:0.05333	validation-logloss:0.38540
[16]	train-logloss:0.04861	validation-logloss:0.37860
[17]	train-logloss:0.04448	validation-logloss:0.38167
[18]	train-logloss:0.04086	validation-

In [29]:
model2 = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=veval, # print losses every ten rounds
    early_stopping_rounds=1 # activate early stopping
)

[0]	train-logloss:0.51791	validation-logloss:0.58426
[1]	train-logloss:0.41573	validation-logloss:0.52100
[2]	train-logloss:0.34269	validation-logloss:0.49901
[3]	train-logloss:0.27600	validation-logloss:0.46496
[4]	train-logloss:0.23084	validation-logloss:0.46228
[5]	train-logloss:0.19491	validation-logloss:0.43869
[6]	train-logloss:0.16608	validation-logloss:0.43089
[7]	train-logloss:0.13953	validation-logloss:0.42889
[8]	train-logloss:0.12093	validation-logloss:0.41069
[9]	train-logloss:0.10580	validation-logloss:0.39907


In [17]:
# cross-validation
results = xgb.cv(
    params=params, 
    dtrain=dtrain,
    num_boost_round=n,
    nfold=5,
    metrics=["logloss", "auc", "error"]
)

In [18]:
results.head()

,train-logloss-mean,train-logloss-std,train-auc-mean,train-auc-std,train-error-mean,train-error-std,test-logloss-mean,test-logloss-std,test-auc-mean,test-auc-std,test-error-mean,test-error-std
0,0.527998,0.008323,0.939569,0.012334,0.0980,0.014089,0.603141,0.006944,0.776672,0.017540,0.284,0.022450
1,0.405826,0.013011,0.991397,0.005998,0.0315,0.010320,0.564429,0.020061,0.776912,0.029266,0.282,0.047074
2,0.331604,0.015128,0.995950,0.002263,0.0290,0.017720,0.535799,0.030611,0.793539,0.033492,0.280,0.026077
3,0.267322,0.011732,0.998926,0.001091,0.0110,0.007517,0.515636,0.034470,0.802633,0.031440,0.256,0.034986
4,0.217024,0.011000,0.999861,0.000278,0.0090,0.006633,0.507862,0.041716,0.808083,0.034415,0.260,0.047749


In [19]:
# hyperparameter tuning with cross-validation

# Validation

In [20]:
preds = model.predict(dval)

In [21]:
print('preds tyep: ', type(preds))
print('preds shape: ', preds.shape)
print('preds:\n',preds)

preds tyep:  <class 'numpy.ndarray'>
preds shape:  (100,)
preds:
 [0.98732704 0.9288599  0.9557566  0.5451001  0.47584033 0.3360091
 0.9935661  0.86174184 0.49405178 0.29101077 0.9143418  0.9815122
 0.9290346  0.9873871  0.9906393  0.2296215  0.17710997 0.31748277
 0.99316907 0.988572   0.9877247  0.14112504 0.9881995  0.9696082
 0.96265197 0.5410623  0.9894628  0.13617423 0.9099501  0.12043644
 0.39894688 0.964254   0.5300371  0.90366435 0.9769682  0.9929409
 0.670899   0.9937115  0.7998163  0.8300792  0.43466127 0.93031186
 0.99357396 0.86708593 0.04462078 0.9808313  0.9801255  0.95304394
 0.7245376  0.67800903 0.40775347 0.77498406 0.16576342 0.987356
 0.9928266  0.97498995 0.92509234 0.8906021  0.985321   0.07395781
 0.9718248  0.15721591 0.4915147  0.26945016 0.11178396 0.22358966
 0.97746265 0.27508605 0.5438842  0.30199614 0.58005863 0.9577517
 0.9595013  0.9265643  0.8284727  0.8130608  0.9082726  0.98409355
 0.9809847  0.7262435  0.9291444  0.96298283 0.9816891  0.65658176
 0.

In [22]:
preds_th = (preds > 0.5).astype(int)
gt_lbls = (val_data['lbls']+1)//2
acc = sum(preds_th == gt_lbls)/len(gt_lbls)
err = sum(preds_th != gt_lbls)/len(gt_lbls)


print('Thresholded preds:\n', preds_th)
print('Ground-truth val. labels:\n', gt_lbls)
print('Val. accuracy: ', acc)
print('Val. error rate: ', err)

Thresholded preds:
 [1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 0]
Ground-truth val. labels:
 [1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 0. 1. 1.]
Val. accuracy:  0.82
Val. error rate:  0.18
